In [1]:
import json
utterance_file = "utterance1.txt"
dialog_cnt = 1000
outfile_cnt = 10
json_list = []
other_domains = ["taxi", "train", "bus", "police", "hotel", "attraction", "hospital"]
service_domain = "restaurant"
turn_id = 0
num_workflows = -1 # track the number of processed workflows

In [2]:
with open(utterance_file) as fp:
    Lines = fp.readlines()

In [3]:
def init_dict(dialog_cnt):
    turn_id = 0
    dialog_cnt += 1
    dialogue_id = "CAiFE" + str(dialog_cnt) + ".json"
    line_dict = {"dialogue_id": dialogue_id, "services": ["restaurant"], 
                 "turns": []}
    return line_dict

In [4]:
def add_other_domains():
    d_list = []
    for domain in other_domains:
        d = {"actions": [], "service": domain, "slots": [], 
            "state": {"active_intent": "NONE", "requested_slots": [], "slot_values": {}}}
        d_list.append(d)
    return d_list

In [5]:
for line in Lines:
    if line[0:4] == "----":
        line_dict = init_dict(dialog_cnt)
        num_workflows += 1 # 0 for the 1st time this loop is executed
        if num_workflows > 0: # append only after at least 1 workflow is processed
            json_list.append(line_dict)
        continue
    
    speaker = line[0: line.index(':')] # identify speaker - letters up to 1st ':'
    #identify if keyword exists in a line and get start and end indices
    ind1 = line.index('[')
    ind2 = line.index(']')
    utterance = line[ind2+3: -2] # extract utterance
    
    # for each turn_id setup the dictionary in turns list
    line_dict["turns"].append({"frames": [], "speaker": speaker, 
                               "turn_id": turn_id, "utterance": utterance}) 
    
    line_lowercase = line.lower() # get the lower case version of the line
    
    # if SYSTEM has attributes fill in; if no attributes then init above is enough 
    if (speaker == "SYSTEM") and (ind2 - ind1 > 1):
        slot_name_val = line_lowercase[ind1+1: ind2] # extract slot name and value pair
        name_list = slot_name_val.split(',')
        slotName = name_list[0].strip()
        slotVal = name_list[1].strip()
        start = line_lowercase.index(slotVal) # get the start index of value
        exclusive_end = start + len(slotVal) # get the end index of value
        frame_dict = {"actions": [], "service": service_domain,
                      "slots": [{"exclusive_end": exclusive_end,
                                 "slot": slotName,
                                 "start": start,
                                 "value": slotVal
                                }
                               ]
                     }
        line_dict["turns"][turn_id]["frames"].append(frame_dict)
        
    if speaker == "USER": # case of speaker being USER
        slot_name_val = line_lowercase[ind1+1: ind2] # extract slot name and value pair
        name_list = slot_name_val.split(',')
        slotName = name_list[0].strip() # 1st value is slot name
        slotVal = name_list[1].strip()  # 2nd value is the slot value
        active_intent = name_list[2].strip() # 3rd is the active intent
        start = line_lowercase.index(slotVal) # get the start index of value
        exclusive_end = start + len(slotVal) # get the end index of value
        frame_dict = {"actions": [], "service": service_domain,
                      "slots": [{"exclusive_end": exclusive_end,
                                 "slot": slotName,
                                 "start": start,
                                 "value": slotVal
                                }
                               ],
                      "state": {"active_intent": active_intent,
                                "requested_slots": [],
                                "slot_values": {}
                               }
                      
                     }
        line_dict["turns"][turn_id]["frames"].append(frame_dict)
        
        other_domain_list = add_other_domains() # get values for other domains
        for val in other_domain_list:
            line_dict["turns"][turn_id]["frames"].append(val)

    turn_id += 1
    
json_list.append(line_dict)

In [6]:
json_object = json.dumps(json_list, indent=2)
outfile_cnt += 1
json_file_name = "dialogues_0" + str(outfile_cnt) + ".json"
with open(json_file_name, "w") as fp:
    fp.write(json_object)